# Наивный Байес

Импортирую нужные библиотеки и загружаю датасет.

In [56]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy

from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

messages = pandas.read_csv('SMSSpamCollection', sep='\t', names=["label", "message"])

In [10]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [47]:
print(messages.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4


Как показывает группировка данных по классам, датасет не сбалансированный, так как спама сильно больше, чем не спама. Если всем новым наблюдениям присваивать класс ham, то точность будет достаточно высокой, но с точки зрения задачи классификации спами это плохое решение, так как такой классификатор вообще не будет отсеивать спам. 

**Балансирую данные**

In [48]:
spam = messages[messages['label'] == 'spam']

In [49]:
spam.head()

,label,message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [62]:
ham = messages[messages['label'] == 'ham']

In [28]:
print(len(ham))

4825


In [61]:
less_ham = ham.sample(n=len(spam))
print(len(less_ham))

747


In [119]:
balanced = pandas.concat([less_ham, spam])
print(balanced.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                 747
      unique                                                732
      top                                Sorry, I'll call later
      freq                                                    4
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4


Теперь соотношение 1:1

## Наивный Байес, фичи – слова

Обучаю модель с дефолтными характеристиками: убираем знаки препинания, не лемматизируем.

In [187]:
bow = CountVectorizer()
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])

In [188]:
# 10 рандомных токенов
list(bow.vocabulary_.keys())[:10]

['gettin',
 'fyi',
 'breathe1',
 'text',
 'hoody',
 'pages',
 'panther',
 '08712101358',
 'chatlines',
 'smiling']

Как видим, по дефолту ничего не лемматизируется.

In [189]:
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [191]:
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.963 0.011


Теперь попробуем оставить знаки препинания, используя токенизатор word_tokenize. Он оставляет знаки препинания.

In [209]:
def tokenize_leave_punct(text): return word_tokenize(text.lower())

bow = CountVectorizer(tokenizer=tokenize_leave_punct)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.954 0.015


Со знаками препинания получилось хуже. Правда, эксперимент был не совсем чистым, потому что токенизаторы вообще разные.

Теперь попробуем лемматизировать.

In [194]:
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

def lemmatize(word):
    noun = lmtzr.lemmatize(word)
    verb = lmtzr.lemmatize(word, 'v')
    if verb != word:
        return verb
    return noun

Attention!
Кусочек кода в клетке выше позаимствован у Тани Гавриловой после того, как мы с ней обсудили следующую проблему (обнаруженную ей). У лемматизатора ворднетлемматайзера nltk есть аргумент, отвечающий за часть речи. По дефолту он считает, что ему подали существительное, и лемматизирует его как существительное. Соответственно, результатом для слова leaves будет leaf, а для going так и останется going.

In [202]:
lmtzr.lemmatize('leaves')

'leaf'

In [203]:
lmtzr.lemmatize('going')

'going'

Естественно, если подать ему аргумент 'v', всё будет иначе.

In [200]:
lmtzr.lemmatize('leaves', 'v')

'leave'

In [201]:
lmtzr.lemmatize('going', 'v')

'go'

Сообветственно, чтобы лемматизация была адекватной, нужно сначала использовать POS тэггер, его выдачу преобразовывать и подавать на вход лемматизатору. У nltk есть pos_tag, но те тэги, который она выдаёт, отличаются от тэгов, которые принимает на вход лемматизатор, причём их много и они разные. Соответственно, можно со всем этим долго возиться и бомбить на тему того, почему нельзя было разу сделать нормальный лемматизатор. А можно воспользоваться тем костылём, что если лемматизация словоформы как глагола даёт какой-то результат, отличный от самой словоформы, то, видимо, это был глагол (или существительное во множественном числе, которое лемматизируется так же). Проблему с leaves это всё же решит плохо, но leaves больше похоже на единичный случай.

In [207]:
def tokenize_lemmatize(text):
    text = word_tokenize(text.lower())
    return [lemmatize(word) for word in text]

In [208]:
bow = CountVectorizer(tokenizer=tokenize_lemmatize)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.954 0.012


Результаты практически не изменились.

In [218]:
def tokenize_lemmatize(text):
    text = word_tokenize(text.lower())
    return [lemmatize(word) for word in text if word not in '!?,.":;']

bow = CountVectorizer(tokenizer=tokenize_lemmatize)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.955 0.014


Без знаков препинания чуууть-чуть получше.

Теперь попробую убирать стоп-слова.

In [221]:
from nltk.corpus import stopwords
STOPWORDS = stopwords.words('english')

def tokenize_lemmatize(text):
    text = word_tokenize(text.lower())
    text = [lemmatize(word) for word in text if word not in '!?,.":;']
    return [word for word in text if word not in STOPWORDS]

bow = CountVectorizer(tokenizer=tokenize_lemmatize)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.944 0.014


Значительно хуже. :/


В общем, кажется, лучше не лемматизировать и убирать знаки препинания (или просто дефолтный токенизатор CountVectorizer работает настолько лучше). А что если использовать его и убирать стоп-слова?

In [256]:
bow = CountVectorizer(stop_words=STOPWORDS)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.95 0.018


Хм, нет, убирать стоп-слова было явно плохой идеей. Хотя встроенный токенизатор и правда работает лучше.

Теперь попробую дефолтные настройки и ограничивать df.

Сверху:

In [232]:
bow = CountVectorizer(max_df=0.3)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.964 0.011


Потрясающе, стало лучше, чем дефолт! (Но не сильно).
Я попробовала несколько разных порогов, лучше всего работает в районе 0.3.

Снизу:

In [243]:
bow = CountVectorizer(min_df=2)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.965 0.01


Лучше всего работает порог 2.

И сверху, и снизу:

In [252]:
bow = CountVectorizer(min_df=2, max_df=0.3)
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])
naive_model = MultinomialNB()
naive_model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(naive_model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.963 0.01


А вот так хуже о_О.

Теперь попробую TfIdfVectorizer с дефолтными настройками.

In [268]:
count_vect = CountVectorizer()
counts = count_vect.fit_transform(balanced['message'])
tfidf_transformer = TfidfTransformer().fit(counts)
tfidf = tfidf_transformer.transform(counts)
naive_model = MultinomialNB()
naive_model.fit(tfidf, balanced['label'])
cv_results = cross_val_score(naive_model, tfidf, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.964 0.009


Кажется, очень даже неплохие результаты. А что если добавить чего-нибудь успешного из прошлых шагов?

In [273]:
count_vect = CountVectorizer(min_df=2)
counts = count_vect.fit_transform(balanced['message'])
tfidf_transformer = TfidfTransformer().fit(counts)
tfidf = tfidf_transformer.transform(counts)
naive_model = MultinomialNB()
naive_model.fit(tfidf, balanced['label'])
cv_results = cross_val_score(naive_model, tfidf, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.957 0.015


Не очень-то. Ну ладно.

**В общем, лучше всего получилось, кажется, при дефолтных настройках CountVectorizer с нижним ограничением на количество вхождений 2 и с применением TfidfTransformer.**

## Деревья решений и рандомный лес.

In [277]:
from sklearn.tree import DecisionTreeClassifier

bow = CountVectorizer()
bow.fit_transform(balanced['message'])
bowed_messages = bow.transform(balanced['message'])

tree = DecisionTreeClassifier(min_samples_split=5)
tree.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(tree, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.924 0.02


Дерево решений работает значительно хуже.

In [278]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(bowed_messages, balanced['label'])
cv_results = cross_val_score(model, bowed_messages, balanced['label'], cv=10, scoring='accuracy')
print(round(cv_results.mean(), 3), round(cv_results.std(), 3))

0.936 0.018


In [ ]:
Рандомный лес